1. Impory necessary libraries and load the dataset.

2. load a pre-trained model, VGG16, excluding the top layers

3. add new layers on top of the base model and compile the model

4. unfreeze some of the layers of the pre-trained model and fine-tune them.


In [5]:
#1. set up environment
! pip install tensorflow
! pip install matplotlib

In [11]:
import numpy as np
import matplotlib as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
# Load the VGG model pre-trained on ImageNet
base_model = VGG16(weights ='imagenet', include_top=False, input_shape=(224,224,3))
# include_top = False meaning allow add custom layer for specific task
# input_layer is 'specific layer value'

# freeze the base model layers
for layer in base_model.layers:
  layer.trainable = False

# meaning is 'freeze the way up basic model's layers not updating during training.
# keep the specific feature during taking traning.

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# create and compile model
model= Sequential([
    base_model,
    Flatten(),
    Dense(256, activation ='relu'),
    Dense(1, activation ='sigmoid') # change the number of classes you have
])

# compile the model
model.compile(optimizer ='adam',loss ='binary_crossentropy', metrics = ['accuracy'])

In [17]:
# creat placeholder images
import os
from PIL import Image
import numpy as np

os.makedirs('sample_data/class_a', exist_ok=True)
# makedirs - make file with way.
os.makedirs('sample_data/class_b', exist_ok=True)

# create image and save
for i in range(10):
  img = Image.fromarray(np.ones((224,224,3),dtype=np.uint8)*255)
  # add 255
  # np.ones is full of this arrary only number '1'
  # full of "1" with [224,224,3] images add 255 -> full of '255' with [224,224,3]
  # 0(drakest colour)-255(brightest colour).   -> the brighter colour -> white

  img.save(f'sample_data/class_a/img_{i}.jpg')

  img = Image.fromarray(np.zeros((224,224,3),dtype=np.uint8))
  # not add 255 -> keep staying '0'
  # meaning is full '0' with [224,224,3] -> black colour
  img.save(f'sample_data/class_b/img_{i}.jpg')

print("sample images created in 'sample_data/'")

sample images created in 'sample_data/'


In [21]:
# train the model
train_datagen = ImageDataGenerator(rescale=1./255) # pixcel value divided with (0~1)
train_generator = train_datagen.flow_from_directory(
    'sample_data/',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary')

# verify if the generator has loaded images correctly
print(f"Found {train_generator.samples} images belonging to {train_generator.num_classes} classes.")

# train the model
if train_generator.samples >0:
  model.fit(train_generator,epochs=10)


Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5000 - loss: 0.7742
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.5000 - loss: 6.2757
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5000 - loss: 3.7992
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.5000 - loss: 0.6041
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.5000 - loss: 0.7248
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.5000 - loss: 0.7357
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.5000 - loss: 0.7424
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.5000 - loss: 0.7447
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.5000 - loss: 0.7432
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.5000 - loss: 0.7387


In [23]:
# fine - tune the model
for layer in base_model.layers[-4:]:
  layer.trainable = True
# 마지막 4개의 레이어를 슬라이싱
# 기본적인 전이학습은  trainable = false 로 가중치 업데이트 불가능인데 여기서 True 로 바꿔줌으로써
# 마지막 4개의 레이어만 학습가능하도록 설정.
# As the result,initial layer got trained with previous specific feature then only last 4 layers has trainied
# for training new proper specific feature. useful of reusing the model more efficient.

# compile the model again
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# fit the model
model.fit(train_generator, epochs =10)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5000 - loss: 0.7322
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - accuracy: 0.5000 - loss: 1.0143
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.5000 - loss: 1.8514
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - accuracy: 0.5000 - loss: 0.6876
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step - accuracy: 0.5000 - loss: 0.7281
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 1.0000 - loss: 0.6226
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.5000 - loss: 0.6524
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step - accuracy: 1.0000 - loss: 0.4593
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.5000 - loss: 0.6471
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step - accuracy: 0.5000 - loss: 0.6817
